# 3D printing a turbulent box

In [ ]:
from pathlib import Path
import sys

import numpy as np
from PIL import Image, ImageOps
from scipy.interpolate import RegularGridInterpolator

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from IPython.display import display

from volrender.image_stack import makeslice, process

plt.style.use([{'image.cmap':'gray_r'}])

Read data. Data is currently (until 01.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/4abe80f1c550806021f85af8c57c886e).

In [ ]:
f = np.load('../data/turbulentbox.npy')
data = f.copy()
del f

## Normalization

Find the largest magnitude of the data values and define a logarithmic norm

In [ ]:
vmax = 10**np.ceil(np.log10(data.max()))
norm = LogNorm(1e-2 * vmax, vmax, clip=True)

## Example plot

Select which slice to plot

In [ ]:
i = 0

In [ ]:
# apply the norm

img_norm = np.array(norm(data[:, :, i]))

# we could now make this straight to an image
im = Image.fromarray(np.uint8(255 - img_norm * 255))
display('Image:', im)

# This is all we need to do to use dithering

im_1 = im.convert("1")
display('Dithered image:', im_1)

## Upscale the data

### Coordinates & aspect ratios

these are the original "coordinates" of the pixels

In [ ]:
x = np.arange(data.shape[0])
y = np.arange(data.shape[1])
z = np.arange(data.shape[2])

create an interpolation function for the 3d data

In [ ]:
f_interp = RegularGridInterpolator((x, y, z), data)

settings of the printer (educated guess, especially the layer thickness might be different)

In [ ]:
height = 10 # this should be the total height of the printed cube in cm

# these are the values according to alphacams website on the J850 Prime
dpi_x = 600
dpi_y = 600
dpi_z = 1200

# these are the values according to fit technologies
# dpi_x = 600
# dpi_y = 300
# dpi_z = 1814 # 0.014 mm layer thickness
# dpi_z = 940 # 0.027 mm layer thickness

# layer_thickness = 2.54 / dpi_z#55e-4 # 14 micron

calculate the new grids in x, y, z

In [ ]:
#n_z = int(height / layer_thickness)
n_z = int(height * dpi_z / 2.54)
n_x = int(n_z / dpi_z * dpi_x)
n_y = int(n_z / dpi_z * dpi_y)

n_x += n_x%2 # add 1 to make it even if it isn't
n_y += n_y%2 # add 1 to make it even if it isn't

x2 = np.linspace(0, data.shape[0] - 1, n_x)
y2 = np.linspace(0, data.shape[1] - 1, n_y)
z2 = np.linspace(0, data.shape[2] - 1, n_z)

this creates the coordinates of the new layer and we'll update the `z` coordinate as we go, interpolating one layer at a time. `x` and `y` stay the same

In [ ]:
coords = np.concatenate((np.meshgrid(x2, y2, z2[0])), axis=-1)

### Iteration

we get the new layer by interpolating the 3D data. We store the images in the path set by `output_dir`.

In [ ]:
output_dir = 'slices'

Prepare output folder

In [ ]:
path = Path(output_dir)

if not path.is_dir():
    path.mkdir()
else:
    files = list(path.glob('slice*.png'))
    if len(files)>0:
        print('directory exists, deleting old files')
        for file in files:
            file.unlink()

select which index in the new z-grid to process

In [ ]:
iz = 0

This cell does the same as `makeslice`: interpolates one layer, creates and dithers the image and writes it to file

In [ ]:
# update coordinates - only last entry changes
n_y, n_x = coords.shape[:-1]
copy = coords.copy()
copy[:, :, -1] = z2[iz]

# interpolate
new_layer = f_interp(copy.reshape([-1, 3])).reshape([n_x, n_y]).T

# normalize, convert to grayscale image
layer_norm = np.array(norm(new_layer))
im = Image.fromarray(np.uint8(255 - layer_norm * 255)).convert('1')

# save as 1bit bitmap
im.save(path / f'slice_{iz:04d}.png', bits=1, optimize=True)

# save the inverted image as well
im_inv = im.convert('L')
im_inv = ImageOps.invert(im_inv)
im_inv = im_inv.convert('1')
im_inv.save(path / f'slice_transp_{iz:04d}.png', bits=1, optimize=True)

this is the same result using `makeslice`

In [ ]:
makeslice(iz, z2, f_interp, coords, norm, path)

## Batch processing

all of the above can also be done in a loop with `process`:
normalizing with the given norm, up-scaling and saving to images. We'll just do this same one here by specifying the `iz` keyword.

In [ ]:
process(data, norm=norm, iz=iz)